In [20]:
import numpy as np
import pandas as pd
import random
import math

In [2]:
class EM(object):
    '''Class for Implementing Expectation Minimization Algorithm'''
    def __init__(self, data, k, d):
        '''
        data = the d dimensional data points
        k = number of clusters
        d = number of dimensions
        '''
        self.X = data
        self.k = k
        self.d = d
        self.N = len(data)
        self.mean = None 
        self.covariance = None
        self.r = [[0 for _ in range(k)] for _ in range(N)]
        self.m_c = [0 for _ in range(k)]
        self.pi_c = [0 for _ in range(k)]
        
    def _initialze(self):
        '''
        Function To Initialize the Expectation Step
        '''
    
    def kmeans(self):
        '''
        Initializes the EM algorithm using KMeans++ initialization stratergy
        yet to complete
        '''
        centers = []
        chosen = {} #hash table to keep track of chosen points
        while (len(centers) != self.k):
            flag = 0
            while(flag == 0):
                n = np.random.randint(0, self.N-1)
                try:
                    flag = chosen[n] - 1
                except:
                    chosen[n] = 1
                    flag = 1
            centers.app
                
            
            
            
    def Gaussian(self, x, mean, covar):
        '''
        Calculates Gaussian Probability
        '''
        num = math.exp(0.5*np.dot(np.dot((x-mean).T, np.linalg.inv(covar)), x-mean))
        den = math.sqrt(2*math.pi*np.linalg.det(covar))
        return num/den
        
    def E_Step(self):
        '''
        Computers ric (confidence score) for each data point
        '''
        for i in range(self.X):
            den = 0
            for j in range(self.k):
                den += self.pi_c[j]*self.Gaussian(self.X[i], self.mean[j], self.covariance[j])
            for c in range(self.k):
                r[i][c] = self.pi_c*self.Gaussian(self.X[i], self.mean[c], self.covariance[c])/den
    
    def M_Step(self):
        '''
        Maximization Step : Updates the Parameters (mean, covariacne, pi_c)
        '''
        for c in range(self.k):
            temp = 0
            mu_temp = 0
            var_temp = 0
            for i in range(self.N):
                temp += self.r[i][c]
                mu_temp += self.r[i][c]*self.X[i]
                var_temp += self.r[i][c]*np.dot((self.X[i]-self.mean[c]).T, self.X[i]-self.mean[c])
            self.m_c[c] = temp
            self.pi_c[c] = temp/self.N
            self.mean[c] = mu_temp/temp
            self.covariance[c] = var_temp/temp    

In [18]:
mean = np.zeros(4)
var  = np.diag(np.ones(4))
np.random.multivariate_normal(mean, var, size = (1, 1))

array([[[-1.16861883,  0.40733924, -0.28855449, -0.26653416]]])